<a href="https://colab.research.google.com/github/Dash400air/Bert_task/blob/main/SST_2ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GPU check

In [ ]:
!nvidia-smi

Mon Sep 20 08:13:41 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.63.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   44C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Setup

In [1]:
!pip install transformers pytorch-lightning textstat

     |████████████████████████████████| 2.8 MB 7.1 MB/s 
     |████████████████████████████████| 923 kB 35.9 MB/s 
     |████████████████████████████████| 101 kB 11.1 MB/s 
     |████████████████████████████████| 52 kB 2.1 MB/s 
     |████████████████████████████████| 3.3 MB 31.0 MB/s 
     |████████████████████████████████| 636 kB 77.8 MB/s 
     |████████████████████████████████| 895 kB 66.3 MB/s 
     |████████████████████████████████| 829 kB 75.0 MB/s 
     |████████████████████████████████| 282 kB 60.4 MB/s 
     |████████████████████████████████| 119 kB 77.8 MB/s 
     |████████████████████████████████| 1.3 MB 71.4 MB/s 
     |████████████████████████████████| 2.0 MB 59.1 MB/s 
     |████████████████████████████████| 142 kB 77.2 MB/s 
     |████████████████████████████████| 294 kB 76.0 MB/s 
  Created wheel for future: filename=future-0.18.2-py3-none-any.whl size=491070 sha256=18a5b0c0a9e9a0726d4371ea149af9d3eb7c504619f9f802356bd2258757f40f
  Stored in directory: /root/.cache/pip

In [2]:
import os
import random

import pandas as pd
import numpy as np

import textstat

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

import pytorch_lightning as pl

from transformers import RobertaTokenizer, RobertaForSequenceClassification

from sklearn.metrics import accuracy_score, recall_score, precision_score
from sklearn.metrics import f1_score, matthews_corrcoef
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold

In [3]:
train = pd.read_csv('/content/drive/MyDrive/GRI/SST-2/SST-2/train.tsv', sep='\t')
test = pd.read_csv('/content/drive/MyDrive/GRI/SST-2/SST-2/dev.tsv', sep='\t')
exam= pd.read_csv('/content/drive/MyDrive/GRI/SST-2/SST-2/test.tsv', sep='\t')

# Max length

In [ ]:
train['label'].value_counts()

1    37569
0    29780
Name: label, dtype: int64

In [ ]:
def get_maxlength(df, columns:list):
    count_df = pd.DataFrame()
    for column in columns:
        texts = df[column].tolist()
        count = []
        for text in texts:
            text = str(text)
            count.append(textstat.lexicon_count(text, removepunct=False))
        count_df[column] = count
    return count_df

count_df = get_maxlength(train, ['sentence'])
count_df.describe()

,sentence
count,67349.000000
mean,9.409553
std,8.073806
min,1.000000
25%,3.000000
50%,7.000000
75%,13.000000
max,52.000000


In [ ]:
count_df = get_maxlength(test, ['sentence'])
count_df.describe()

,sentence
count,872.000000
mean,19.548165
std,8.763900
min,2.000000
25%,13.000000
50%,19.000000
75%,26.000000
max,47.000000


In [ ]:
count_df = get_maxlength(exam, ['sentence'])
count_df.describe()

,sentence
count,1821.000000
mean,19.233937
std,8.922386
min,2.000000
25%,12.000000
50%,18.000000
75%,25.000000
max,56.000000


# Config

In [4]:
class Config:
    def __init__(self):
        self.model = 'roberta-base'
        self.tokenizer = RobertaTokenizer.from_pretrained(self.model)
        self.sc = RobertaForSequenceClassification
        self.text_max = 35
        self.seed = 42

Config = Config()

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/481 [00:00<?, ?B/s]

# Seed

In [5]:
def seed_torch(seed=42):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True

seed_torch(Config.seed)

# Kfold

In [ ]:
def get_train_data(train):
    Fold = StratifiedKFold(n_splits=5, shuffle=True, random_state=Config.seed)
    for n, (train_index, val_index) in enumerate(Fold.split(train, train["label"])):
        train.loc[val_index, "fold"] = int(n)
    train["fold"] = train["fold"].astype(np.uint8)

    return train

train = get_train_data(train)

In [ ]:
train.head()

,sentence,label,fold
0,hide new secretions from the parental units,0,2
1,"contains no wit , only labored gags",0,4
2,that loves its characters and communicates som...,1,2
3,remains utterly satisfied to remain the same t...,0,1
4,on the worst revenge-of-the-nerds clichés the ...,0,4


# Dataset

In [ ]:
class BaseDataset(Dataset):
    def __init__(self, df):
        self.df = df

        self.sentence = df['sentence'].tolist()
        self.labels = df['label']

        self.tokenizer = Config.tokenizer
        self.encoded = self.tokenizer(self.sentence,
                                 return_tensors='pt',
                                 max_length = Config.text_max,
                                 padding='max_length',
                                 truncation = True, 
                                 add_special_tokens=True,
                                 return_attention_mask=True
                                 )

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        input_ids = self.encoded['input_ids'][idx]
        attention_mask = self.encoded['attention_mask'][idx]
        label = torch.tensor(self.labels[idx])
        output = {'input_ids': input_ids, 'attention_mask': attention_mask, 'labels': label}
        return output

# Dataloader

In [ ]:
def get_dataloader(train, fold):
    trn_idx = train[train["fold"] != fold].index
    val_idx = train[train["fold"] == fold].index

    train_folds = train.loc[trn_idx].reset_index(drop=True)
    valid_folds = train.loc[val_idx].reset_index(drop=True)

    train_dataset = BaseDataset(train_folds)
    valid_dataset = BaseDataset(valid_folds)

    train_loader = DataLoader(
            train_dataset,
            batch_size=32,
            shuffle=True,
            drop_last=True,
        )
    valid_loader = DataLoader(
            valid_dataset,
            batch_size=32,
            shuffle=False,
        )
    return train_loader, valid_loader

def get_testloader(test):
  test_dataset = BaseDataset(test)
  test_loader = DataLoader(
          test_dataset,
          batch_size=32,
          shuffle=False,
      )
  return test_loader

# Model

In [6]:
class SequenceClassification_pl(pl.LightningModule):

  def __init__(self, model_name, num_labels, lr):
    # model_name: Transformersのモデル名
    # num_labels: ラベルの数
    # lr: 学習率

    super().__init__()

    self.save_hyperparameters()
    
    self.bert_sc = Config.sc.from_pretrained(
        model_name,
        num_labels=num_labels
    )

  def forward(self, **inputs):
    return self.bert_sc(**inputs)

  def training_step(self, batch, batch_idx):
    output = self.bert_sc(**batch)
    loss = output.loss
    self.log('train_loss', loss)
    return loss

  def validation_step(self, batch, batch_idx):
    output = self.bert_sc(**batch)
    val_loss = output.loss
    self.log('val_loss', val_loss)

  def test_step(self, batch, batch_idx):
    labels = batch.pop('labels').detach().cpu().numpy() #labelsについて，GPU上のTensorではfbeta_scoreが受け付けてくれないため，CPUに移動させ，Numpyに変換
    output = self.bert_sc(**batch)
    labels_predicted = output.logits.detach().cpu().numpy().argmax(-1)  #同上

    accuracy = accuracy_score(labels, labels_predicted)
    recall = recall_score(labels, labels_predicted)
    precision = precision_score(labels, labels_predicted)
    f1 = f1_score(labels, labels_predicted)
    mcc = matthews_corrcoef(labels, labels_predicted)
    
    values = {'accuracy': accuracy, 'recall': recall,
              'precision': precision, 'f1': f1, 'matthews_corrcoef': mcc}
    self.log_dict(values)
  
  def configure_optimizers(self):
    optimizer = torch.optim.AdamW(self.parameters(), lr=self.hparams.lr)
    scheduler = {'scheduler': torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer), 
                 'monitor': 'val_loss'}
    return [optimizer], scheduler

# Run

In [ ]:
model = SequenceClassification_pl(
    Config.model, 
    num_labels=2, 
    lr=2e-5)

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'roberta.pooler.dense.bias', 'roberta.pooler.dense.weight', 'lm_head.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.weight', 'classifie

In [ ]:
checkpoint = pl.callbacks.ModelCheckpoint(
    monitor='val_loss',
    mode='min',
    save_top_k=1,
    save_weights_only=True,
    dirpath='model/',
)

early_stopping = pl.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=3,
    mode='min'
)

trainer = pl.Trainer(
    gpus=-1,
    max_epochs=10,
    callbacks = [checkpoint, early_stopping]
)

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/callbacks/model_checkpoint.py:446: UserWarning: Checkpoint directory model/ exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


In [ ]:
dataloader_train, dataloader_val = get_dataloader(train, 1)

In [ ]:
trainer.fit(model, dataloader_train, dataloader_val)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name    | Type                             | Params
-------------------------------------------------------------
0 | bert_sc | RobertaForSequenceClassification | 124 M 
-------------------------------------------------------------
124 M     Trainable params
0         Non-trainable params
124 M     Total params
498.589   Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

Training: -1it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

# Test

In [ ]:
dataloader_test = get_testloader(test)

In [ ]:
result = trainer.test(test_dataloaders=dataloader_test)

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/trainer.py:679: LightningDeprecationWarning: `trainer.test(test_dataloaders)` is deprecated in v1.4 and will be removed in v1.6. Use `trainer.test(dataloaders)` instead.
  "`trainer.test(test_dataloaders)` is deprecated in v1.4 and will be removed in v1.6."
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: 0it [00:00, ?it/s]

--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'accuracy': 0.9311926364898682,
 'f1': 0.9291689395904541,
 'matthews_corrcoef': 0.8620530366897583,
 'precision': 0.9301403164863586,
 'recall': 0.9335532784461975}
--------------------------------------------------------------------------------


# Examination

In [7]:
best_model = '/content/drive/MyDrive/GRI/SST-2/epoch=2-step=5048.ckpt'
predict_model = SequenceClassification_pl.load_from_checkpoint(best_model)
bert_sc = predict_model.bert_sc.cuda()

Downloading:   0%|          | 0.00/501M [00:00<?, ?B/s]

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.weight', 'roberta.pooler.dense.weight', 'lm_head.bias', 'lm_head.decoder.weight', 'roberta.pooler.dense.bias', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.out_proj.weight', 'classi

In [16]:
def examine(texts):
    encoded = Config.tokenizer(
        texts,
        padding='longest',
        return_tensors='pt'
    )

    encoded = {k: v.cuda() for k, v in encoded.items()}

    with torch.no_grad():
      output = bert_sc(**encoded)
    label_predicted = output.logits.detach().cpu().numpy().argmax(-1)
    
    print('--------RESULT---------')
    for text, label in zip(texts, label_predicted.tolist()):
        print(f'"{text}"')
        if label:
            print('Positive')
        else:
            print('Negative')
        print('----------------------')

**速度**\
データ数：1821\
エンコード＋推測時間：7秒

---


1データあたり3.8(ms)

In [ ]:
exam.shape

(1821, 2)

In [ ]:
examine(exam['sentence'].tolist())

In [17]:
texts = ['I love apple.',
         'I hate apple.',
         'I think so.',
         'I disagree.']

examine(texts)

--------RESULT---------
"I love apple."
Positive
----------------------
"I hate apple."
Negative
----------------------
"I think so."
Positive
----------------------
"I disagree."
Negative
----------------------
